# Bases de datos

In [1]:
import pandas as pd
#import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import openpyxl
import pandas.io.formats.excel 
import time
%matplotlib inline
#pd.set_option('display.notebook_repr_html', True)
#pd.set_option('display.max_columns', 11)
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.width', 78)
pd.set_option('precision', 3)
pd.options.display.float_format = '{:.5f}'.format


In [2]:
data=pd.read_excel('annual_generation_state.xls','TOTALES',index_col=0)
data=data.loc[data['GENERATION (Megawatthours)']>0,:]

In [3]:
years=np.unique(data.index)

In [4]:
states=np.unique(data['STATE'])
states

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'US-TOTAL', 'US-Total', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'],
      dtype=object)

In [5]:
energies=np.unique(data['ENERGY SOURCE'])
energies

array(['Coal', 'Geothermal', 'Hydroelectric Conventional', 'Natural Gas',
       'Nuclear', 'Other', 'Other Biomass', 'Other Gases', 'Petroleum',
       'Pumped Storage', 'Solar Thermal and Photovoltaic', 'Wind',
       'Wood and Wood Derived Fuels'], dtype=object)

In [6]:
total_by_states=list(map(lambda s:list(map(lambda y:sum(data.loc[data['STATE']==states[s],'GENERATION (Megawatthours)'].loc[data.loc[data['STATE']==states[s],:].index==years[y]]),range(len(years)))),range(len(states))))

In [7]:
data_by_states=list(map(lambda e:data.loc[data['STATE']==states[e],:],range(len(states))))

In [8]:
data_by_states=list(map(lambda s:list(map(lambda y:(data.loc[data['STATE']==states[s],:].loc[data.loc[data['STATE']==states[s],:].index==years[y]]),range(len(years)))),range(len(states))))

In [9]:
#state,year
proportions_by_states=list(map(lambda s:list(map(lambda y:pd.concat([data_by_states[s][y],data_by_states[s][y]['GENERATION (Megawatthours)']/total_by_states[s][y]],axis=1),range(len(years)))),range(len(states))))

In [10]:
#state,type of energy
df=list(map(lambda s:list(map(lambda e:pd.DataFrame(np.reshape((np.column_stack(list(map(lambda y:pd.DataFrame(np.ones([1,4])*.0001) if (proportions_by_states[s][y].loc[proportions_by_states[s][y]['ENERGY SOURCE']==energies[e],:]).empty else proportions_by_states[s][y].loc[proportions_by_states[s][y]['ENERGY SOURCE']==energies[e],:],range(len(years)))))),(28,4))),range(len(energies)))),range(len(states))))

In [11]:
#state,type of energy
df_with_ret=list(map(lambda s:list(map(lambda e:pd.concat([df[s][e],(df[s][e][3]/df[s][e][3].shift(1)-1)*100],axis=1),range(len(energies)))),range(len(states))))

In [12]:
df_with_rets=list(map(lambda s:list(map(lambda e:pd.concat([df_with_ret[s][e],(df[s][e][2]/df[s][e][2].shift(1)-1)*100],axis=1),range(len(energies)))),range(len(states))))

In [13]:
df_with_rets[0][0]

,0,1,2,3,3,2
0,AK,Coal,510573.00000,0.09118,NaN,NaN
1,AK,Coal,520514.00000,0.09657,5.91303,1.94703
2,AK,Coal,495795.00000,0.09357,-3.10577,-4.74896
3,AK,Coal,533123.00000,0.09318,-0.41842,7.52892
4,AK,Coal,502866.00000,0.08428,-9.55241,-5.67543
5,AK,Coal,528930.00000,0.08777,4.14280,5.18309
6,AK,Coal,428736.00000,0.06995,-20.30203,-18.94277
7,AK,Coal,442702.00000,0.07193,2.82205,3.25748
8,AK,Coal,524613.00000,0.09000,25.13025,18.50251
9,AK,Coal,523369.00000,0.09020,0.21914,-0.23713


In [64]:
doc=openpyxl.load_workbook('data.xlsx')
hoja=doc.get_sheet_by_name('Hoja1')
list(map(lambda s:list(map(lambda e:list(map(lambda f: hoja.append([""]) if pd.DataFrame.equals(df_with_rets[s][e],df_with_rets[0][5]) else hoja.append([df_with_rets[s][e].index[f],df_with_rets[s][e].iloc[f,0],df_with_rets[s][e].iloc[f,1],df_with_rets[s][e].iloc[f,2],df_with_rets[s][e].iloc[f,3],df_with_rets[s][e].iloc[f,4],df_with_rets[s][e].iloc[f,5]]),range(28))),range(len(energies)))),range(len(states))))
doc.save('data.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  
